In [77]:
import pandas as pd
import numpy as np

In [78]:
df = pd.read_csv('cleaned_data.csv')
df = df[['Support', 'Resistance', 'Hammer', 'Williams%R', 'ZeroCross', 'SignalCross', 'Decision14','Decision28']]
df = df.loc[df["Decision28"] != -1] # we exclude the value with a decision of -1
df = df.drop('Decision14', axis =1)

In [79]:
def cal_prior(df, Y):

    classes = sorted(list(df[Y].unique()))
    prior = []

    for i in classes:
        prior.append(len(df[df[Y] == i]) / len(df[Y]))
    
    return prior

In [80]:
def cal_likelihood(df, feature_name, feature_value, Y, label):

    df = df[df[Y] == label]
    mean = df[feature_name].mean()
    std = df[feature_name].std()

    p_x_given_y = len(df[df[feature_name] == feature_value]) / len(df)

    return p_x_given_y

In [81]:
def naive_bayes(df, X, Y):

    features = list(df.columns)[:-1]
    prior = cal_prior(df, Y)

    Y_pred = []

    for x in X:
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for i in range(len(labels)):
            for j in range(len(features)):
                likelihood[i] *= cal_likelihood(df, features[j], x[j], Y, labels[i])
        
        # Calculate posterior probability (we ignore denominator)
        post_prob = [1]*len(labels)
        for i in range(len(labels)):
            post_prob[i] = likelihood[i] * prior[i]
        
        Y_pred.append(np.argmax(post_prob))

    return np.array(Y_pred)

In [91]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.294, random_state=40, shuffle = False)

X_test = test.iloc[:, :-1].values
Y_test = test.iloc[:, -1].values
Y_pred = naive_bayes(train, X=X_test, Y='Decision28')

from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import balanced_accuracy_score
print(confusion_matrix(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='micro'))
print(balanced_accuracy_score(Y_test, Y_pred))

[[ 12 117]
 [  5 155]]
0.5778546712802768
0.5308866279069767


## Comparing results with pre-built models
### taken on https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html#sklearn.naive_bayes.CategoricalNB

In [93]:
from sklearn.naive_bayes import CategoricalNB

In [94]:
classifier = CategoricalNB()
classifier.fit(train.iloc[:,:-1].values, train.iloc[:,-1].values)

CategoricalNB()

In [95]:
Y_pred2 = classifier.predict(test.iloc[:, :-1].values)

In [96]:
print(confusion_matrix(Y_test, Y_pred2))
print(f1_score(Y_test, Y_pred2, average='micro'))

[[ 12 117]
 [  5 155]]
0.5778546712802768


In [97]:
classifier.score(test.iloc[:, :-1].values,Y_test)

0.5778546712802768

In [98]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB

In [99]:
mnb = MultinomialNB()
mnb.fit(train.iloc[:,:-1].values, train.iloc[:,-1].values)
mnb.predict(test.iloc[:, :-1].values)
print(mnb.score(test.iloc[:, :-1].values,Y_test))
print(balanced_accuracy_score(Y_test, mnb.predict(test.iloc[:, :-1].values)))

0.5813148788927336
0.5347625968992248


In [100]:
cnb = ComplementNB()
cnb.fit(train.iloc[:,:-1].values, train.iloc[:,-1].values)
cnb.predict(test.iloc[:, :-1].values)
print(cnb.score(test.iloc[:, :-1].values,Y_test))
print(balanced_accuracy_score(Y_test, cnb.predict(test.iloc[:, :-1].values)))

0.532871972318339
0.5646075581395349


In [101]:
pred = mnb.predict(test.iloc[:, :-1].values)

In [102]:
pred2 = cnb.predict(test.iloc[:, :-1].values)
print(confusion_matrix(Y_test,pred2))

[[111  18]
 [117  43]]


In [103]:
print(confusion_matrix(Y_test,pred))

[[ 13 116]
 [  5 155]]


In [104]:
mnb.classes_

array([0, 1], dtype=int64)

In [105]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1], dtype=int64)

In [106]:
test

,Support,Resistance,Hammer,Williams%R,ZeroCross,SignalCross,Decision28
725,0,0,0.0,1,0.0,1.0,1
726,0,0,0.0,1,0.0,1.0,1
727,0,0,0.0,1,0.0,1.0,1
728,0,0,0.0,1,1.0,1.0,1
729,0,0,0.0,1,1.0,1.0,1
...,...,...,...,...,...,...,...
1009,0,0,0.0,0,1.0,1.0,0
1010,0,0,0.0,2,0.0,0.0,0
1011,0,0,0.0,2,0.0,0.0,0
1012,1,0,0.0,2,0.0,0.0,0
